# Analysis of Normalized Queries

This notebook contains an analysis on CIViC evidence data

In [1]:
from pathlib import Path
from enum import Enum
import zipfile

import pandas as pd
from civicpy import civic as civicpy
import plotly.express as px

In [2]:
# Use latest cache that has been pushed to the repo
latest_cache_zip_path = sorted(Path().glob("../cache-*.pkl.zip"))[-1]
print(f"Using {latest_cache_zip_path} for civicpy cache")

with zipfile.ZipFile(latest_cache_zip_path, "r") as zip_ref:
    zip_ref.extractall("../")

civicpy.load_cache(local_cache_path=Path("../cache.pkl"), on_stale="ignore")

Using ../cache-20230803.pkl.zip for civicpy cache


True

## Total Variants in CIViC

In [3]:
civic_variant_ids = civicpy.get_all_variants()
total_number_variants = len(civic_variant_ids)
f"Total Number of variants in CIViC: {total_number_variants}"

'Total Number of variants in CIViC: 3582'

## Total Evidence items in CIViC

Need to remove all rejected evidence items

In [4]:
civic_evidence_ids = civicpy.get_all_evidence(include_status=["accepted", "submitted"])

In [5]:
total_ac_sub_evidence = len(civic_evidence_ids)
f"Total Number of accepted and submitted evidence items in CIViC: {total_ac_sub_evidence}"

'Total Number of accepted and submitted evidence items in CIViC: 9920'

## Total Molecular Profiles in CIViC

In [6]:
civic_molprof_ids = civicpy.get_all_molecular_profiles(include_status=["accepted", "submitted", "rejected"])

# Create analysis functions / global variables 

In [7]:
class VariantNormType(str, Enum):
    """Variation Normalization types"""

    NORMALIZED = "Normalized"
    UNABLE_TO_NORMALIZE = "Unable to Normalize"
    NOT_SUPPORTED = "Not Supported"


VARIANT_NORM_TYPE_VALUES = [v.value for v in VariantNormType.__members__.values()]

In [8]:
class VariantCategory(str, Enum):
    """Create enum for the kind of variants that are in CIViC."""
    EXPRESSION = "Expression Variants"
    EPIGENETIC_MODIFICATION = "Epigenetic Modification"
    FUSION = "Fusion Variants"
    SEQUENCE_VARS = "Sequence Variants"
    GENE_FUNC = "Gene Function Variants"
    REARRANGEMENTS = "Rearrangement Variants"
    COPY_NUMBER = "Copy Number Variants"
    OTHER = "Other Variants"
    GENOTYPES = "Genotype Variants"
    REGION_DEFINED_VAR = "Region Defined Variants"
    TRANSCRIPT_VAR = "Transcript Variants"  # no attempt to normalize these ones, since there is no query we could use


VARIANT_CATEGORY_VALUES = [v.value for v in VariantCategory.__members__.values()]

## Summary dicts 

These dictionaries will be mutated and used at the end of the analysis

In [9]:
variant_analysis_summary = {
    "Variant Category": VARIANT_NORM_TYPE_VALUES,
    "Fraction of all CIViC Variant Items": [],
    "Percentage of all CIViC Variant Items": [],
    "Fraction of Accepted Variant Items": [],
    "Percentage of Accepted Variant Items": [],
    "Fraction of Not Accepted Variant Items": [],
    "Percentage of Not Accepted Variant Items": []
}
variant_analysis_summary

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Fraction of all CIViC Variant Items': [],
 'Percentage of all CIViC Variant Items': [],
 'Fraction of Accepted Variant Items': [],
 'Percentage of Accepted Variant Items': [],
 'Fraction of Not Accepted Variant Items': [],
 'Percentage of Not Accepted Variant Items': []}

In [10]:
evidence_analysis_summary = {
    "Variant Category": VARIANT_NORM_TYPE_VALUES,
    "Fraction of all CIViC Evidence Items": [],
    "Percentage of all CIViC Evidence Items": [],
    "Fraction of Accepted Evidence Items": [],
    "Percentage of Accepted Evidence Items": [],
    "Fraction of Submitted Evidence Items": [],
    "Percentage of Submitted Evidence Items": []
}
evidence_analysis_summary

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Fraction of all CIViC Evidence Items': [],
 'Percentage of all CIViC Evidence Items': [],
 'Fraction of Accepted Evidence Items': [],
 'Percentage of Accepted Evidence Items': [],
 'Fraction of Submitted Evidence Items': [],
 'Percentage of Submitted Evidence Items': []}

In [11]:
def variant_analysis(df: pd.DataFrame, variant_norm_type: VariantNormType) -> pd.DataFrame:
    """Do variant analysis (counts, percentages)

    :param df: Dataframe of variants
    :param variant_norm_type: The kind of variants that are in `df`
    :return: Transformed dataframe with variant ID duplicates dropped
    """
    # Drop duplicate rows
    df = df.drop_duplicates(subset=["variant_id"])
    variant_ids = list(df["variant_id"])

    # Count
    num_variants = len(variant_ids)
    fraction_variants = f"{num_variants} / {total_number_variants}"
    print(f"\nNumber of {variant_norm_type.value} Variants in CIViC: {fraction_variants}")

    # Percentage
    percentage_variants = f"{num_variants / total_number_variants * 100:.2f}%"
    print(f"Percentage of {variant_norm_type.value} Variants in CIViC: {percentage_variants}")

    # Get accepted counts
    num_accepted_variants = df.variant_accepted.sum()
    fraction_accepted_variants = f"{num_accepted_variants} / {num_variants}"
    print(f"\nNumber of accepted {variant_norm_type.value} Variants: {fraction_accepted_variants}")

    # Get accepted percentage
    percentage_accepted_variants = f"{num_accepted_variants / num_variants * 100:.2f}%"
    print(f"Percentage of accepted {variant_norm_type.value} Variants: {percentage_accepted_variants}")

    # Get not accepted counts
    num_not_accepted_variants = len(df) - num_accepted_variants
    fraction_not_accepted_variants = f"{num_not_accepted_variants} / {num_variants}"
    print(f"\nNumber of not accepted {variant_norm_type.value} Variants: {fraction_not_accepted_variants}")

    # Get not accepted percentage
    percentage_not_accepted_variants = f"{num_not_accepted_variants / num_variants * 100:.2f}%"
    print(f"Percentage of not accepted {variant_norm_type.value} Variants: {percentage_not_accepted_variants}")

    variant_analysis_summary["Fraction of all CIViC Variant Items"].append(fraction_variants)
    variant_analysis_summary["Percentage of all CIViC Variant Items"].append(percentage_variants)
    variant_analysis_summary["Fraction of Accepted Variant Items"].append(fraction_accepted_variants)
    variant_analysis_summary["Percentage of Accepted Variant Items"].append(percentage_accepted_variants)
    variant_analysis_summary["Fraction of Not Accepted Variant Items"].append(fraction_not_accepted_variants)
    variant_analysis_summary["Percentage of Not Accepted Variant Items"].append(percentage_not_accepted_variants)

    return df

In [12]:
def transform_df_evidence_ids(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include evidence ID information

    :param df: Dataframe of variants
    :return: Transformed dataframe with evidence ID information
    """
    tmp_df = df.copy(deep=True)

    _variants_evidence_ids = []
    variant_ids = list(tmp_df["variant_id"])

    for v_id in variant_ids:
        _variant_evidence_ids = []

        for variant in civic_variant_ids:
            if int(v_id) == variant.id:
                for mp in variant.molecular_profiles:
                    for e in mp.evidence_items:
                        if e.id not in _variant_evidence_ids:
                            _variant_evidence_ids.append(e.id)

        _variants_evidence_ids.append(_variant_evidence_ids or "")

    tmp_df["evidence_ids"] = _variants_evidence_ids

    # Explode and rename evidence ids field
    tmp_df = tmp_df.explode(column="evidence_ids")
    tmp_df = tmp_df.rename(columns={"evidence_ids": "evidence_id"})

    return tmp_df

In [13]:
def transform_df_evidence(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include evidence status, rating, and level

    :param df: Dataframe of variants
    :return: Transformed dataframe with evidence status, rating, and level information.
        Rejected evidence items will be dropped.
    """
    variants_evidence_ids = list(df["evidence_id"])

    # Add evidence status, rating, and level information
    _variants_evidence_statuses = []
    _variants_evidence_ratings = []
    _variants_evidence_levels = []

    for eid in variants_evidence_ids:
        _variant_evidence_statuses = []
        _variant_evidence_ratings = []
        _variant_evidence_levels = []

        for evidence in civic_evidence_ids:
            if int(eid) == evidence.id:
                if evidence.status not in _variant_evidence_statuses:
                    _variant_evidence_statuses.append(evidence.status)

                if evidence.rating not in _variant_evidence_ratings:
                    _variant_evidence_ratings.append(evidence.rating)

                if evidence.evidence_level not in _variant_evidence_levels:
                    _variant_evidence_levels.append(evidence.evidence_level)

        _variants_evidence_statuses.append(_variant_evidence_statuses or "")
        _variants_evidence_ratings.append(_variant_evidence_ratings or "")
        _variants_evidence_levels.append(_variant_evidence_levels or "")

    df["evidence_status"] = _variants_evidence_statuses
    df["evidence_status"] = df["evidence_status"].str.join(", ")
    df["evidence_rating"] = _variants_evidence_ratings
    df["evidence_level"] = _variants_evidence_levels

    # Drop rejected evidence items
    df = df.drop(df[df.evidence_status == "rejected"].index)

    return df


In [14]:
def evidence_analysis(df: pd.DataFrame, variant_norm_type: VariantNormType) -> pd.DataFrame:
    """Do evidence analysis (counts, percentages)

    :param df: Dataframe of variants
    :param variant_norm_type: The kind of variants that are in `df`
    :return: Transformed dataframe with evidence ID duplicates dropped
    """
    # Count
    num_variant_unique_evidence_items = len(set(df.evidence_id))
    fraction_evidence_items = f"{num_variant_unique_evidence_items} / {total_ac_sub_evidence}"
    print(f"Number of {variant_norm_type.value} Variant Evidence items in CIViC: {fraction_evidence_items}")

    # Percentage
    percentage_evidence_items = f"{num_variant_unique_evidence_items / total_ac_sub_evidence * 100:.2f}%"
    print(f"Percentage of {variant_norm_type.value} Variant Evidence items in CIViC: {percentage_evidence_items}")

    # Add evidence accepted column
    df["evidence_accepted"] = df.evidence_status.map({"accepted": True, "submitted": False})

    # Drop evidence id duplicates
    df = df.drop_duplicates(subset=["evidence_id"])

    # Get accepted counts
    num_accepted_evidences_variants = df.evidence_accepted.sum()
    fraction_accepted_evidences_variants = f"{num_accepted_evidences_variants} / {num_variant_unique_evidence_items}"
    print(f"\nNumber of accepted {variant_norm_type.value} Variant Evidence items: {fraction_accepted_evidences_variants}")

    # Get accepted percentage
    percentage_accepted_evidences_variants = f"{num_accepted_evidences_variants / num_variant_unique_evidence_items * 100:.2f}%"
    print(f"Percentage of accepted {variant_norm_type.value} Variant Evidence items: {percentage_accepted_evidences_variants}")

    # Get submitted counts
    number_submitted_evidences_variants = len(df) - num_accepted_evidences_variants
    fraction_submitted_evidences_variants = f"{number_submitted_evidences_variants} / {num_variant_unique_evidence_items}"
    print(f"\nNumber of submitted {variant_norm_type.value} Variant Evidence items: {fraction_submitted_evidences_variants}")

    # Get submitted percentage
    percentage_submitted_evidences_variants = f"{number_submitted_evidences_variants / num_variant_unique_evidence_items * 100:.2f}%"
    print(f"Percentage of not submitted {variant_norm_type.value} Variant Evidence items: {percentage_submitted_evidences_variants}")

    evidence_analysis_summary["Fraction of all CIViC Evidence Items"].append(fraction_evidence_items)
    evidence_analysis_summary["Percentage of all CIViC Evidence Items"].append(percentage_evidence_items)
    evidence_analysis_summary["Fraction of Accepted Evidence Items"].append(fraction_accepted_evidences_variants)
    evidence_analysis_summary["Percentage of Accepted Evidence Items"].append(percentage_accepted_evidences_variants)
    evidence_analysis_summary["Fraction of Submitted Evidence Items"].append(fraction_submitted_evidences_variants)
    evidence_analysis_summary["Percentage of Submitted Evidence Items"].append(percentage_submitted_evidences_variants)

    return df

In [15]:
def transform_df_mp_id(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile ID information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile ID information
    """
    tmp_df = df.copy(deep=True)

    variants_molprof_ids = []
    variant_ids = list(tmp_df["variant_id"])

    for v_id in variant_ids:
        variant_molprof_ids = []

        for variant in civic_variant_ids:
            if int(v_id) == variant.id:
                for mp in variant.molecular_profiles:
                    if mp.id not in variant_molprof_ids:
                        variant_molprof_ids.append(mp.id)

        variants_molprof_ids.append(variant_molprof_ids or "")

    tmp_df["molecular_profile_id"] = variants_molprof_ids
    return tmp_df

In [16]:
def transform_df_mp_score(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile score information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile score information
    """
    variants_molprof_scores = []
    normalized_variant_molprof_ids = list(df["molecular_profile_id"])

    for mp_ids in normalized_variant_molprof_ids:
        variant_molprof_scores = []
        for mp_id in mp_ids:
            for molprof in civic_molprof_ids:
                if int(mp_id) == molprof.id:
                    if molprof.molecular_profile_score not in variant_molprof_scores:
                        variant_molprof_scores.append(molprof.molecular_profile_score)

        variants_molprof_scores.append(variant_molprof_scores or "")

    df["molecular_profile_score"] = variants_molprof_scores
    return df

In [17]:
def transform_df_mp_score_sum(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile score sum information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile score sum information
    """
    df["molecular_profile_score_sum"] = df["molecular_profile_score"].apply(lambda x: sum(x))
    return df

## List of Normalized Variants ID's

In [18]:
normalized_queries_df = pd.read_csv("../variation_analysis/able_to_normalize_queries.csv", sep="\t")
normalized_queries_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize
1,1988,NC_000003.11:g.10191649A>T,genomic,True,Stop Lost,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,normalize
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize
3,1986,NC_000003.11:g.10191648G>T,genomic,True,Stop Lost,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,normalize
4,1987,NC_000003.11:g.10191649A>G,genomic,True,Stop Lost,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,normalize


## Variant analysis

In [19]:
normalized_queries_df = variant_analysis(normalized_queries_df, VariantNormType.NORMALIZED)
normalized_queries_df.head()


Number of Normalized Variants in CIViC: 1876 / 3582
Percentage of Normalized Variants in CIViC: 52.37%

Number of accepted Normalized Variants: 869 / 1876
Percentage of accepted Normalized Variants: 46.32%

Number of not accepted Normalized Variants: 1007 / 1876
Percentage of not accepted Normalized Variants: 53.68%


,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize
1,1988,NC_000003.11:g.10191649A>T,genomic,True,Stop Lost,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,normalize
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize
3,1986,NC_000003.11:g.10191648G>T,genomic,True,Stop Lost,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,normalize
4,1987,NC_000003.11:g.10191649A>G,genomic,True,Stop Lost,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,normalize


In [20]:
variant_analysis_summary

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Fraction of all CIViC Variant Items': ['1876 / 3582'],
 'Percentage of all CIViC Variant Items': ['52.37%'],
 'Fraction of Accepted Variant Items': ['869 / 1876'],
 'Percentage of Accepted Variant Items': ['46.32%'],
 'Fraction of Not Accepted Variant Items': ['1007 / 1876'],
 'Percentage of Not Accepted Variant Items': ['53.68%']}

## Transform df for evidence analysis

In [21]:
normalized_queries_add_evidence_df = transform_df_evidence_ids(normalized_queries_df)
normalized_queries_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,evidence_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,9347
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,6724
1,1988,NC_000003.11:g.10191649A>T,genomic,True,Stop Lost,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,normalize,5336
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,10779
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,6723


In [22]:
normalized_queries_add_evidence_df = transform_df_evidence(normalized_queries_add_evidence_df)
normalized_queries_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,evidence_id,evidence_status,evidence_rating,evidence_level
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,9347,submitted,[3],[C]
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,6724,accepted,[2],[C]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,Stop Lost,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,normalize,5336,accepted,[2],[C]
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,10779,submitted,[3],[C]
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,6723,accepted,[2],[C]


## Evidence analysis

In [23]:
normalized_queries_add_evidence_df = evidence_analysis(normalized_queries_add_evidence_df, VariantNormType.NORMALIZED)
normalized_queries_add_evidence_df.head()

Number of Normalized Variant Evidence items in CIViC: 6005 / 9920
Percentage of Normalized Variant Evidence items in CIViC: 60.53%

Number of accepted Normalized Variant Evidence items: 2080 / 6005
Percentage of accepted Normalized Variant Evidence items: 34.64%

Number of submitted Normalized Variant Evidence items: 3925 / 6005
Percentage of not submitted Normalized Variant Evidence items: 65.36%


,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,9347,submitted,[3],[C],False
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,6724,accepted,[2],[C],True
1,1988,NC_000003.11:g.10191649A>T,genomic,True,Stop Lost,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,normalize,5336,accepted,[2],[C],True
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,10779,submitted,[3],[C],False
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,6723,accepted,[2],[C],True


## Impact
molecular profile score

### Import molecular profile id

In [24]:
normalized_queries_add_molprof_df = transform_df_mp_id(normalized_queries_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,molecular_profile_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,[2362]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,Stop Lost,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,normalize,[1864]
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,[2361]
3,1986,NC_000003.11:g.10191648G>T,genomic,True,Stop Lost,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,normalize,[1862]
4,1987,NC_000003.11:g.10191649A>G,genomic,True,Stop Lost,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,normalize,[1863]


In [25]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,molecular_profile_id
86,190,EGFR Amplification,protein,True,Transcript Amplification,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,normalize,"[190, 4175, 4346, 4567]"


### Import molecular profile scores

In [26]:
normalized_queries_add_molprof_df = transform_df_mp_score(normalized_queries_add_molprof_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,molecular_profile_id,molecular_profile_score
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,[2362],[5.0]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,Stop Lost,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,normalize,[1864],[5.0]
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,[2361],[5.0]
3,1986,NC_000003.11:g.10191648G>T,genomic,True,Stop Lost,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,normalize,[1862],[10.0]
4,1987,NC_000003.11:g.10191649A>G,genomic,True,Stop Lost,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,normalize,[1863],[5.0]


In [27]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,molecular_profile_id,molecular_profile_score
86,190,EGFR Amplification,protein,True,Transcript Amplification,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,normalize,"[190, 4175, 4346, 4567]","[173.0, 5.0, 0.0]"


In [28]:
normalized_queries_add_molprof_df = transform_df_mp_score_sum(normalized_queries_add_molprof_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,Stop Lost,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,normalize,[2362],[5.0],5.0
1,1988,NC_000003.11:g.10191649A>T,genomic,True,Stop Lost,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,normalize,[1864],[5.0],5.0
2,2488,3-10191647-T-G,genomic,True,Stop Lost,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,normalize,[2361],[5.0],5.0
3,1986,NC_000003.11:g.10191648G>T,genomic,True,Stop Lost,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,normalize,[1862],[10.0],10.0
4,1987,NC_000003.11:g.10191649A>G,genomic,True,Stop Lost,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,normalize,[1863],[5.0],5.0


In [29]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190]

,variant_id,query,query_type,variant_accepted,civic_variant_types,vrs_id,succeeded_endpoint,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
86,190,EGFR Amplification,protein,True,Transcript Amplification,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,normalize,"[190, 4175, 4346, 4567]","[173.0, 5.0, 0.0]",178.0


# Analysis of Unable to Normalize Queries

## List of Unable to Normalize Variant ID's

In [30]:
not_normalized_queries_df = pd.read_csv("../variation_analysis/unable_to_normalize_queries.csv", sep="\t")
not_normalized_queries_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,exception_raised,message,warnings
0,748,MLH1 *757L,protein,True,Stop Lost,False,unable to normalize,['Unable to tokenize: *757L']
1,3718,AR A748V,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A748V']
2,3725,AR A765T,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A765T']
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t..."
4,248,TERT C228T,protein,True,Regulatory Region Variant,False,unable to normalize,['Unable to translate TERT C228T']


## Variant analysis

In [31]:
not_normalized_queries_df = variant_analysis(not_normalized_queries_df, VariantNormType.UNABLE_TO_NORMALIZE)
not_normalized_queries_df.head()


Number of Unable to Normalize Variants in CIViC: 80 / 3582
Percentage of Unable to Normalize Variants in CIViC: 2.23%

Number of accepted Unable to Normalize Variants: 11 / 80
Percentage of accepted Unable to Normalize Variants: 13.75%

Number of not accepted Unable to Normalize Variants: 69 / 80
Percentage of not accepted Unable to Normalize Variants: 86.25%


,variant_id,query,query_type,variant_accepted,civic_variant_types,exception_raised,message,warnings
0,748,MLH1 *757L,protein,True,Stop Lost,False,unable to normalize,['Unable to tokenize: *757L']
1,3718,AR A748V,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A748V']
2,3725,AR A765T,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A765T']
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t..."
4,248,TERT C228T,protein,True,Regulatory Region Variant,False,unable to normalize,['Unable to translate TERT C228T']


## Transform df for evidence analysis

In [32]:
not_normalized_quer_add_evidence_df = transform_df_evidence_ids(not_normalized_queries_df)
not_normalized_quer_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,exception_raised,message,warnings,evidence_id
0,748,MLH1 *757L,protein,True,Stop Lost,False,unable to normalize,['Unable to tokenize: *757L'],1812
1,3718,AR A748V,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A748V'],10128
2,3725,AR A765T,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A765T'],10135
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...",11488
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...",11494


In [33]:
not_normalized_quer_add_evidence_df = transform_df_evidence(not_normalized_quer_add_evidence_df)
not_normalized_quer_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,exception_raised,message,warnings,evidence_id,evidence_status,evidence_rating,evidence_level
0,748,MLH1 *757L,protein,True,Stop Lost,False,unable to normalize,['Unable to tokenize: *757L'],1812,accepted,[1],[C]
1,3718,AR A748V,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A748V'],10128,submitted,[3],[D]
2,3725,AR A765T,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A765T'],10135,submitted,[3],[D]
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...",11488,,,
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...",11494,submitted,[4],[D]


## Evidence analysis

In [34]:
not_normalized_quer_add_evidence_df = evidence_analysis(not_normalized_quer_add_evidence_df, VariantNormType.UNABLE_TO_NORMALIZE)
not_normalized_quer_add_evidence_df.head()

Number of Unable to Normalize Variant Evidence items in CIViC: 129 / 9920
Percentage of Unable to Normalize Variant Evidence items in CIViC: 1.30%

Number of accepted Unable to Normalize Variant Evidence items: 17 / 129
Percentage of accepted Unable to Normalize Variant Evidence items: 13.18%

Number of submitted Unable to Normalize Variant Evidence items: 112 / 129
Percentage of not submitted Unable to Normalize Variant Evidence items: 86.82%


,variant_id,query,query_type,variant_accepted,civic_variant_types,exception_raised,message,warnings,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,748,MLH1 *757L,protein,True,Stop Lost,False,unable to normalize,['Unable to tokenize: *757L'],1812,accepted,[1],[C],True
1,3718,AR A748V,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A748V'],10128,submitted,[3],[D],False
2,3725,AR A765T,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A765T'],10135,submitted,[3],[D],False
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...",11488,,,,NaN
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...",11494,submitted,[4],[D],False


## Impact
molecular profile score

### Import molecular profile id

In [35]:
not_normalized_queries_add_molprof_df = transform_df_mp_id(not_normalized_queries_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,exception_raised,message,warnings,molecular_profile_id
0,748,MLH1 *757L,protein,True,Stop Lost,False,unable to normalize,['Unable to tokenize: *757L'],[729]
1,3718,AR A748V,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A748V'],[3586]
2,3725,AR A765T,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A765T'],[3593]
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...","[4463, 4472]"
4,248,TERT C228T,protein,True,Regulatory Region Variant,False,unable to normalize,['Unable to translate TERT C228T'],[244]


### Import molecular profile scores

In [36]:
not_normalized_queries_add_molprof_df = transform_df_mp_score(not_normalized_queries_add_molprof_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,exception_raised,message,warnings,molecular_profile_id,molecular_profile_score
0,748,MLH1 *757L,protein,True,Stop Lost,False,unable to normalize,['Unable to tokenize: *757L'],[729],[2.5]
1,3718,AR A748V,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A748V'],[3586],[0.0]
2,3725,AR A765T,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A765T'],[3593],[0.0]
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...","[4463, 4472]",[0.0]
4,248,TERT C228T,protein,True,Regulatory Region Variant,False,unable to normalize,['Unable to translate TERT C228T'],[244],[40.0]


In [37]:
not_normalized_queries_add_molprof_df = transform_df_mp_score_sum(not_normalized_queries_add_molprof_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,civic_variant_types,exception_raised,message,warnings,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,748,MLH1 *757L,protein,True,Stop Lost,False,unable to normalize,['Unable to tokenize: *757L'],[729],[2.5],2.5
1,3718,AR A748V,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A748V'],[3586],[0.0],0.0
2,3725,AR A765T,protein,False,Not provided,False,unable to normalize,['Unable to translate AR A765T'],[3593],[0.0],0.0
3,4485,ERBB2 A775_G776ins YVMA,protein,False,Not provided,False,unable to normalize,"['Unable to tokenize: A775_G776ins', 'Unable t...","[4463, 4472]",[0.0],0.0
4,248,TERT C228T,protein,True,Regulatory Region Variant,False,unable to normalize,['Unable to translate TERT C228T'],[244],[40.0],40.0


# Analysis of Not Supported Variants

### List of Not Supported Variant ID's

In [38]:
not_supported_queries_df = pd.read_csv("../variation_analysis/not_supported_variants.csv", sep="\t")
not_supported_queries_df.head()

,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted
0,4170,VHL,NaN,Not provided,Transcript Variants,False
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False
2,4214,VHL,NaN,Not provided,Transcript Variants,False
3,4216,VHL,NaN,Not provided,Transcript Variants,False
4,4278,VHL,NaN,Not provided,Transcript Variants,False


## Variant Analysis

In [39]:
not_supported_queries_df = variant_analysis(not_supported_queries_df, VariantNormType.NOT_SUPPORTED)
not_supported_queries_df.head()


Number of Not Supported Variants in CIViC: 1563 / 3582
Percentage of Not Supported Variants in CIViC: 43.63%

Number of accepted Not Supported Variants: 790 / 1563
Percentage of accepted Not Supported Variants: 50.54%

Number of not accepted Not Supported Variants: 773 / 1563
Percentage of not accepted Not Supported Variants: 49.46%


,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted
0,4170,VHL,NaN,Not provided,Transcript Variants,False
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False
2,4214,VHL,NaN,Not provided,Transcript Variants,False
3,4216,VHL,NaN,Not provided,Transcript Variants,False
4,4278,VHL,NaN,Not provided,Transcript Variants,False


### Not Supported Variant Analysis by Subcategory 

In [40]:
not_supported_variant_analysis_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "Fraction of Not Supported Variant Items": [],
    "Percent of Not Supported Variant Items": [],
    "Fraction of all CIViC Variant Items": [],
    "Percent of all CIViC Variant Items": [],
    "Fraction of Accepted Variant Items": [],
    "Percent of Accepted Variant Items": [],
    "Fraction of Not Accepted Variant Items": [],
    "Percent of Not Accepted Variant Items": []
}

In [41]:
not_supported_variant_categories_summary_data = dict()
total_number_unique_not_supported_variants = len(set(not_supported_queries_df.variant_id))

for category in VARIANT_CATEGORY_VALUES:  # These are not supported categories
    not_supported_variant_categories_summary_data[category] = {}
    category_df = not_supported_queries_df[not_supported_queries_df.category == category]

    # Count
    number_unique_not_supported_category_variants = len(set(category_df.variant_id))
    not_supported_variant_categories_summary_data[category]["number_unique_not_supported_category_variants"] = number_unique_not_supported_category_variants

    # Fraction
    fraction_not_supported_category_variant_of_civic = f"{number_unique_not_supported_category_variants} / {total_number_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_supported_category_variant_of_civic"] = fraction_not_supported_category_variant_of_civic

    # Percent
    percent_not_supported_category_variant_of_civic = f"{number_unique_not_supported_category_variants / total_number_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percent_not_supported_category_variant_of_civic"] = percent_not_supported_category_variant_of_civic

    # Not supported fraction
    fraction_not_supported_category_variant_of_total_not_supported = f"{number_unique_not_supported_category_variants} / {total_number_unique_not_supported_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_supported_category_variant_of_total_not_supported"] = fraction_not_supported_category_variant_of_total_not_supported

    # Not supported percent
    percent_not_supported_category_variant_of_total_not_supported = f"{number_unique_not_supported_category_variants / total_number_unique_not_supported_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percent_not_supported_category_variant_of_total_not_supported"] = percent_not_supported_category_variant_of_total_not_supported

    # Accepted fraction
    number_accepted_not_supported_category_variants = category_df.variant_accepted.sum()
    fraction_accepted_not_supported_category_variants = f"{number_accepted_not_supported_category_variants} / {number_unique_not_supported_category_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_accepted_not_supported_category_variants"] = fraction_accepted_not_supported_category_variants

    # Accepted percent
    percentage_accepted_not_supported_category_variants = f"{number_accepted_not_supported_category_variants / number_unique_not_supported_category_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percentage_accepted_not_supported_category_variants"] = percentage_accepted_not_supported_category_variants

    # Not accepted fraction
    number_not_accepted_not_supported_category_variants = len(category_df) - number_accepted_not_supported_category_variants
    fraction_not_accepted_not_supported_category_variants = f" {number_not_accepted_not_supported_category_variants} / {number_unique_not_supported_category_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_accepted_not_supported_category_variants"] = fraction_not_accepted_not_supported_category_variants

    # Not accepted percent
    percentage_not_accepted_not_supported_category_variants = f"{number_not_accepted_not_supported_category_variants / number_unique_not_supported_category_variants * 100:.2f}%"
    not_supported_variant_categories_summary_data[category]["percentage_not_accepted_not_supported_category_variants"] = percentage_not_accepted_not_supported_category_variants

    not_supported_variant_analysis_summary["Fraction of all CIViC Variant Items"].append(fraction_not_supported_category_variant_of_civic)
    not_supported_variant_analysis_summary["Percent of all CIViC Variant Items"].append(percent_not_supported_category_variant_of_civic)
    not_supported_variant_analysis_summary["Fraction of Not Supported Variant Items"].append(fraction_not_supported_category_variant_of_total_not_supported)
    not_supported_variant_analysis_summary["Percent of Not Supported Variant Items"].append(percent_not_supported_category_variant_of_total_not_supported)
    not_supported_variant_analysis_summary["Fraction of Accepted Variant Items"].append(fraction_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Percent of Accepted Variant Items"].append(percentage_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Fraction of Not Accepted Variant Items"].append(fraction_not_accepted_not_supported_category_variants)
    not_supported_variant_analysis_summary["Percent of Not Accepted Variant Items"].append(percentage_not_accepted_not_supported_category_variants)

## Transform df for evidence analysis

In [42]:
not_supported_variants_add_evidence_df = transform_df_evidence_ids(not_supported_queries_df)
not_supported_variants_add_evidence_df.head()

,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted,evidence_id
0,4170,VHL,NaN,Not provided,Transcript Variants,False,10647
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False,7428
2,4214,VHL,NaN,Not provided,Transcript Variants,False,10752
3,4216,VHL,NaN,Not provided,Transcript Variants,False,10754
4,4278,VHL,NaN,Not provided,Transcript Variants,False,10958


In [43]:
not_supported_variants_add_evidence_df = transform_df_evidence(not_supported_variants_add_evidence_df)
not_supported_variants_add_evidence_df.head()

,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level
0,4170,VHL,NaN,Not provided,Transcript Variants,False,10647,submitted,[2],[C]
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False,7428,submitted,[3],[C]
2,4214,VHL,NaN,Not provided,Transcript Variants,False,10752,submitted,[3],[C]
3,4216,VHL,NaN,Not provided,Transcript Variants,False,10754,submitted,[3],[C]
4,4278,VHL,NaN,Not provided,Transcript Variants,False,10958,submitted,[3],[C]


## Evidence analysis

In [44]:
not_supported_variants_add_evidence_df = evidence_analysis(not_supported_variants_add_evidence_df, VariantNormType.NOT_SUPPORTED)
not_supported_variants_add_evidence_df.head()

Number of Not Supported Variant Evidence items in CIViC: 4340 / 9920
Percentage of Not Supported Variant Evidence items in CIViC: 43.75%

Number of accepted Not Supported Variant Evidence items: 2227 / 4340
Percentage of accepted Not Supported Variant Evidence items: 51.31%

Number of submitted Not Supported Variant Evidence items: 2113 / 4340
Percentage of not submitted Not Supported Variant Evidence items: 48.69%


,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,4170,VHL,NaN,Not provided,Transcript Variants,False,10647,submitted,[2],[C],False
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False,7428,submitted,[3],[C],False
2,4214,VHL,NaN,Not provided,Transcript Variants,False,10752,submitted,[3],[C],False
3,4216,VHL,NaN,Not provided,Transcript Variants,False,10754,submitted,[3],[C],False
4,4278,VHL,NaN,Not provided,Transcript Variants,False,10958,submitted,[3],[C],False


### Not Supported Variant Evidence Analysis by Subcategory 

 List all the possible variant categories, have to use non unique file since evidence items are used more than once across groups


In [45]:
not_supported_variant_categories = not_supported_variants_add_evidence_df.category.unique()
[v for v in not_supported_variant_categories]

['Transcript Variants',
 'Fusion Variants',
 'Rearrangement Variants',
 'Sequence Variants',
 'Region Defined Variants',
 'Other Variants',
 'Copy Number Variants',
 'Gene Function Variants',
 'Expression Variants',
 'Genotype Variants',
 'Epigenetic Modification']

Evidence ID's can qualify for more than one kind of variant category

In [46]:
duplicate = not_supported_variants_add_evidence_df[not_supported_variants_add_evidence_df.duplicated("evidence_id", keep=False)]
duplicate

,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted


In [47]:
not_supported_variant_evidence_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "Fraction of CIViC Evidence Items": [],
    "Percent of CIViC Evidence Items": [],
    "Fraction of Not Supported Variant Evidence Items": [],
    "Percent of Not Supported Variant Evidence Items": [],
    "Fraction of Accepted Evidence Items": [],
    "Percent of Accepted Evidence Items": [],
    "Fraction of Submitted Evidence Items": [],
    "Percent of Submitted Evidence Items": []
}

In [48]:
not_supported_variant_categories_evidence_summary_data = dict()
total_number_not_supported_variant_unique_evidence_items = len(set(not_supported_variants_add_evidence_df.evidence_id))

for category in VARIANT_CATEGORY_VALUES:
    not_supported_variant_categories_evidence_summary_data[category] = {}
    evidence_category_df = not_supported_variants_add_evidence_df[not_supported_variants_add_evidence_df.category == category]
    evidence_category_df = evidence_category_df.drop_duplicates(subset=["evidence_id"])

    # Count
    number_unique_not_supported_category_evidence = len(set(evidence_category_df.evidence_id))
    not_supported_variant_categories_evidence_summary_data[category]["number_unique_not_supported_category_evidence"] = number_unique_not_supported_category_evidence

    # Fraction
    fraction_not_supported_category_variant_evidence_of_civic = f"{number_unique_not_supported_category_evidence} / {total_ac_sub_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_not_supported_category_variant_evidence_of_civic"] = fraction_not_supported_category_variant_evidence_of_civic

    # Percent
    percent_not_supported_category_variant_evidence_of_civic = f"{number_unique_not_supported_category_evidence / total_ac_sub_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percent_not_supported_category_variant_evidence_of_civic"] = percent_not_supported_category_variant_evidence_of_civic

    # Not supported fraction
    fraction_not_supported_category_variant_evidence_of_total_not_supported = f"{number_unique_not_supported_category_evidence} / {total_number_not_supported_variant_unique_evidence_items}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_not_supported_category_variant_evidence_of_total_not_supported"] = fraction_not_supported_category_variant_evidence_of_total_not_supported

    # Not supported percent
    percent_not_supported_category_variant_evidence_of_total_not_supported = f"{number_unique_not_supported_category_evidence / total_number_not_supported_variant_unique_evidence_items * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percent_not_supported_category_variant_evidence_of_total_not_supported"] = percent_not_supported_category_variant_evidence_of_total_not_supported

    # Accepted fraction
    number_accepted_not_supported_category_variant_evidence = evidence_category_df.evidence_accepted.sum()
    fraction_accepted_evidence_not_supported_category_variants = f"{number_accepted_not_supported_category_variant_evidence} / {number_unique_not_supported_category_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_accepted_evidence_not_supported_category_variants"] = fraction_accepted_evidence_not_supported_category_variants

    # Accepted percent
    percentage_accepted_evidence_not_supported_category_variants = f"{number_accepted_not_supported_category_variant_evidence / number_unique_not_supported_category_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percentage_accepted_evidence_not_supported_category_variants"] = percentage_accepted_evidence_not_supported_category_variants

    # Submitted fraction
    number_submitted_not_supported_category_variant_evidence = number_unique_not_supported_category_evidence - evidence_category_df.evidence_accepted.sum()
    fraction_submitted_evidence_not_supported_category_variants = f"{number_submitted_not_supported_category_variant_evidence} / {number_unique_not_supported_category_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_submitted_evidence_not_supported_category_variants"] = fraction_submitted_evidence_not_supported_category_variants

    # Submitted percent
    percentage_submitted_evidence_not_supported_category_variants = f"{number_submitted_not_supported_category_variant_evidence / number_unique_not_supported_category_evidence * 100:.2f}%"
    not_supported_variant_categories_evidence_summary_data[category]["percentage_submitted_evidence_not_supported_category_variants"] = percentage_submitted_evidence_not_supported_category_variants

    not_supported_variant_evidence_summary["Fraction of CIViC Evidence Items"].append(fraction_not_supported_category_variant_evidence_of_civic)
    not_supported_variant_evidence_summary["Percent of CIViC Evidence Items"].append(percent_not_supported_category_variant_evidence_of_civic)
    not_supported_variant_evidence_summary["Fraction of Not Supported Variant Evidence Items"].append(fraction_not_supported_category_variant_evidence_of_total_not_supported)
    not_supported_variant_evidence_summary["Percent of Not Supported Variant Evidence Items"].append(percent_not_supported_category_variant_evidence_of_total_not_supported)
    not_supported_variant_evidence_summary["Fraction of Accepted Evidence Items"].append(fraction_accepted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Percent of Accepted Evidence Items"].append(percentage_accepted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Fraction of Submitted Evidence Items"].append(fraction_submitted_evidence_not_supported_category_variants)
    not_supported_variant_evidence_summary["Percent of Submitted Evidence Items"].append(percentage_submitted_evidence_not_supported_category_variants)

## Impact
molecular profile score

### Import molecular profile id

In [49]:
not_supported_variants_add_molprof_df = transform_df_mp_id(not_supported_queries_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted,molecular_profile_id
0,4170,VHL,NaN,Not provided,Transcript Variants,False,[4038]
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False,[4350]
2,4214,VHL,NaN,Not provided,Transcript Variants,False,[4082]
3,4216,VHL,NaN,Not provided,Transcript Variants,False,[4084]
4,4278,VHL,NaN,Not provided,Transcript Variants,False,[4146]


### Import molecular profile scores

In [50]:
not_supported_variants_add_molprof_df = transform_df_mp_score(not_supported_variants_add_molprof_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted,molecular_profile_id,molecular_profile_score
0,4170,VHL,NaN,Not provided,Transcript Variants,False,[4038],[0.0]
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False,[4350],[0.0]
2,4214,VHL,NaN,Not provided,Transcript Variants,False,[4082],[0.0]
3,4216,VHL,NaN,Not provided,Transcript Variants,False,[4084],[0.0]
4,4278,VHL,NaN,Not provided,Transcript Variants,False,[4146],[0.0]


In [51]:
not_supported_variants_add_molprof_df = transform_df_mp_score_sum(not_supported_variants_add_molprof_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,civic_variant_types,category,variant_accepted,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,4170,VHL,NaN,Not provided,Transcript Variants,False,[4038],[0.0],0.0
1,4417,ALK,FBXO11::ALK,Not provided,Fusion Variants,False,[4350],[0.0],0.0
2,4214,VHL,NaN,Not provided,Transcript Variants,False,[4082],[0.0],0.0
3,4216,VHL,NaN,Not provided,Transcript Variants,False,[4084],[0.0],0.0
4,4278,VHL,NaN,Not provided,Transcript Variants,False,[4146],[0.0],0.0


### Impact by Subcategory

In [52]:
not_supported_impact_summary = {
    "Category": VARIANT_CATEGORY_VALUES,
    "Total Sum Impact Score": [],
    "Total Number Evidence Items": [v["number_unique_not_supported_category_evidence"] for v in not_supported_variant_categories_evidence_summary_data.values()],
    "% Accepted Evidence Items": [v["percentage_accepted_evidence_not_supported_category_variants"] for v in not_supported_variant_categories_evidence_summary_data.values()],
    "Total Number Variants": [v["number_unique_not_supported_category_variants"] for v in not_supported_variant_categories_summary_data.values()]
}

In [53]:
not_supported_variant_categories_impact_data = dict()
for category in VARIANT_CATEGORY_VALUES:
    not_supported_variant_categories_impact_data[category] = {}
    impact_category_df = not_supported_variants_add_molprof_df[not_supported_variants_add_molprof_df.category == category]

    total_sum_not_supported_category_impact = impact_category_df["molecular_profile_score_sum"].sum()
    not_supported_variant_categories_impact_data[category]["total_sum_not_supported_category_impact"] = total_sum_not_supported_category_impact

    not_supported_impact_summary["Total Sum Impact Score"].append(total_sum_not_supported_category_impact)

    print(f"{category}: {total_sum_not_supported_category_impact}")

Expression Variants: 3618.0
Epigenetic Modification: 285.5
Fusion Variants: 6558.75
Sequence Variants: 2746.75
Gene Function Variants: 1805.5
Rearrangement Variants: 2794.0
Copy Number Variants: 225.0
Other Variants: 653.5
Genotype Variants: 312.5
Region Defined Variants: 6199.5
Transcript Variants: 356.5


In [54]:
not_supported_variant_impact_df = pd.DataFrame(not_supported_impact_summary)

# Summary

## Variant Analysis

In [55]:
all_variant_df = pd.DataFrame(variant_analysis_summary)

In [56]:
all_variant_df["Percentage of all CIViC Variant Items"] = all_variant_df["Fraction of all CIViC Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of all CIViC Variant Items"] + ")"
all_variant_df["Percentage of Accepted Variant Items"] = all_variant_df["Fraction of Accepted Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of Accepted Variant Items"] + ")"
all_variant_df["Percentage of Not Accepted Variant Items"] = all_variant_df["Fraction of Not Accepted Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of Not Accepted Variant Items"] + ")"

In [57]:
all_variant_df = all_variant_df.drop(["Fraction of all CIViC Variant Items", "Fraction of Accepted Variant Items", "Fraction of Not Accepted Variant Items"], axis=1)

In [58]:
all_variant_percent_of_civic_df = all_variant_df.drop(["Percentage of Accepted Variant Items", "Percentage of Not Accepted Variant Items"], axis=1)
all_variant_percent_evidence_df = all_variant_df.drop(["Percentage of all CIViC Variant Items"], axis=1)

Summary Table 1: The table below shows the 3 categories that CIViC variants were divided into after normalization and what percentage they make up of all variants in CIViC data. 

In [59]:
all_variant_percent_of_civic_df = all_variant_percent_of_civic_df.set_index("Variant Category")
all_variant_percent_of_civic_df

,Percentage of all CIViC Variant Items
Variant Category,
Normalized,1876 / 3582 (52.37%)
Unable to Normalize,80 / 3582 (2.23%)
Not Supported,1563 / 3582 (43.63%)


In [60]:
summary_table_1 = all_variant_percent_of_civic_df

Summary Table 2: The table below shows the 3 categories that CIViC variants were divided into after normalization and what percentage of the variants in each category are accepted (have at least one evidence item that is accepted) or not.

In [61]:
all_variant_percent_evidence_df = all_variant_percent_evidence_df.set_index("Variant Category")
all_variant_percent_evidence_df

,Percentage of Accepted Variant Items,Percentage of Not Accepted Variant Items
Variant Category,,
Normalized,869 / 1876 (46.32%),1007 / 1876 (53.68%)
Unable to Normalize,11 / 80 (13.75%),69 / 80 (86.25%)
Not Supported,790 / 1563 (50.54%),773 / 1563 (49.46%)


In [62]:
summary_table_2 = all_variant_percent_evidence_df

In [63]:
not_supported_variant_df = pd.DataFrame(not_supported_variant_analysis_summary)
not_supported_variant_df

,Category,Fraction of Not Supported Variant Items,Percent of Not Supported Variant Items,Fraction of all CIViC Variant Items,Percent of all CIViC Variant Items,Fraction of Accepted Variant Items,Percent of Accepted Variant Items,Fraction of Not Accepted Variant Items,Percent of Not Accepted Variant Items
0,Expression Variants,287 / 1563,18.36%,287 / 3582,8.01%,180 / 287,62.72%,107 / 287,37.28%
1,Epigenetic Modification,14 / 1563,0.90%,14 / 3582,0.39%,14 / 14,100.00%,0 / 14,0.00%
2,Fusion Variants,294 / 1563,18.81%,294 / 3582,8.21%,204 / 294,69.39%,90 / 294,30.61%
3,Sequence Variants,133 / 1563,8.51%,133 / 3582,3.71%,73 / 133,54.89%,60 / 133,45.11%
4,Gene Function Variants,91 / 1563,5.82%,91 / 3582,2.54%,49 / 91,53.85%,42 / 91,46.15%
5,Rearrangement Variants,116 / 1563,7.42%,116 / 3582,3.24%,47 / 116,40.52%,69 / 116,59.48%
6,Copy Number Variants,34 / 1563,2.18%,34 / 3582,0.95%,19 / 34,55.88%,15 / 34,44.12%
7,Other Variants,83 / 1563,5.31%,83 / 3582,2.32%,42 / 83,50.60%,41 / 83,49.40%
8,Genotype Variants,16 / 1563,1.02%,16 / 3582,0.45%,12 / 16,75.00%,4 / 16,25.00%
9,Region Defined Variants,129 / 1563,8.25%,129 / 3582,3.60%,99 / 129,76.74%,30 / 129,23.26%


In [64]:
not_supported_variant_df["Percent of Not Supported Variant Items"] = not_supported_variant_df["Fraction of Not Supported Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Not Supported Variant Items"] + ")"
not_supported_variant_df["Percent of all CIViC Variant Items"] = not_supported_variant_df["Fraction of all CIViC Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of all CIViC Variant Items"] + ")"
not_supported_variant_df["Percent of Accepted Variant Items"] = not_supported_variant_df["Fraction of Accepted Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Accepted Variant Items"] + ")"
not_supported_variant_df["Percent of Not Accepted Variant Items"] = not_supported_variant_df["Fraction of Not Accepted Variant Items"].astype(str) + "  (" + not_supported_variant_df["Percent of Not Accepted Variant Items"] + ")"

In [65]:
not_supported_variant_df = not_supported_variant_df.drop(["Fraction of Not Supported Variant Items", "Fraction of all CIViC Variant Items", "Fraction of Accepted Variant Items", "Fraction of Not Accepted Variant Items"], axis=1)
not_supported_variant_df

,Category,Percent of Not Supported Variant Items,Percent of all CIViC Variant Items,Percent of Accepted Variant Items,Percent of Not Accepted Variant Items
0,Expression Variants,287 / 1563 (18.36%),287 / 3582 (8.01%),180 / 287 (62.72%),107 / 287 (37.28%)
1,Epigenetic Modification,14 / 1563 (0.90%),14 / 3582 (0.39%),14 / 14 (100.00%),0 / 14 (0.00%)
2,Fusion Variants,294 / 1563 (18.81%),294 / 3582 (8.21%),204 / 294 (69.39%),90 / 294 (30.61%)
3,Sequence Variants,133 / 1563 (8.51%),133 / 3582 (3.71%),73 / 133 (54.89%),60 / 133 (45.11%)
4,Gene Function Variants,91 / 1563 (5.82%),91 / 3582 (2.54%),49 / 91 (53.85%),42 / 91 (46.15%)
5,Rearrangement Variants,116 / 1563 (7.42%),116 / 3582 (3.24%),47 / 116 (40.52%),69 / 116 (59.48%)
6,Copy Number Variants,34 / 1563 (2.18%),34 / 3582 (0.95%),19 / 34 (55.88%),15 / 34 (44.12%)
7,Other Variants,83 / 1563 (5.31%),83 / 3582 (2.32%),42 / 83 (50.60%),41 / 83 (49.40%)
8,Genotype Variants,16 / 1563 (1.02%),16 / 3582 (0.45%),12 / 16 (75.00%),4 / 16 (25.00%)
9,Region Defined Variants,129 / 1563 (8.25%),129 / 3582 (3.60%),99 / 129 (76.74%),30 / 129 (23.26%)


In [66]:
not_supported_variant_percent_of_civic_df = not_supported_variant_df.drop(["Percent of Not Supported Variant Items", "Percent of Accepted Variant Items", "Percent of Not Accepted Variant Items"], axis=1)
not_supported_variant_percent_of_not_supported_df = not_supported_variant_df.drop(["Percent of all CIViC Variant Items", "Percent of Accepted Variant Items", "Percent of Not Accepted Variant Items"], axis=1)
not_supported_variant_percent_evidence_df = not_supported_variant_df.drop(["Percent of all CIViC Variant Items", "Percent of Not Supported Variant Items"], axis=1)

Summary Table 3: The table below shows the categories that the Not Supported variants were broken into and what percentage of all CIViC variants they make up. These percentages will not add up to 100% because Not Supported variants make up 45.62% of all CIViC variants. 

In [67]:
not_supported_variant_percent_of_civic_df = not_supported_variant_percent_of_civic_df.set_index("Category")
not_supported_variant_percent_of_civic_df

,Percent of all CIViC Variant Items
Category,
Expression Variants,287 / 3582 (8.01%)
Epigenetic Modification,14 / 3582 (0.39%)
Fusion Variants,294 / 3582 (8.21%)
Sequence Variants,133 / 3582 (3.71%)
Gene Function Variants,91 / 3582 (2.54%)
Rearrangement Variants,116 / 3582 (3.24%)
Copy Number Variants,34 / 3582 (0.95%)
Other Variants,83 / 3582 (2.32%)
Genotype Variants,16 / 3582 (0.45%)


In [68]:
summary_table_3 = not_supported_variant_percent_of_civic_df

Summary Table 4: The table below shows the Not Supported variants broken up into 12 sub categories and what percent each sub category take up in Not Supported variant group.

In [69]:
not_supported_variant_percent_of_not_supported_df = not_supported_variant_percent_of_not_supported_df.set_index("Category")
not_supported_variant_percent_of_not_supported_df

,Percent of Not Supported Variant Items
Category,
Expression Variants,287 / 1563 (18.36%)
Epigenetic Modification,14 / 1563 (0.90%)
Fusion Variants,294 / 1563 (18.81%)
Sequence Variants,133 / 1563 (8.51%)
Gene Function Variants,91 / 1563 (5.82%)
Rearrangement Variants,116 / 1563 (7.42%)
Copy Number Variants,34 / 1563 (2.18%)
Other Variants,83 / 1563 (5.31%)
Genotype Variants,16 / 1563 (1.02%)


In [70]:
summary_table_4: not_supported_variant_percent_of_not_supported_df

Summary Table 5: The table below shows the percent of variant items in each Not Supported variant sub category that are accepted(have at least one evidence item that is accepted) or not, per category.

In [71]:
not_supported_variant_percent_evidence_df = not_supported_variant_percent_evidence_df.set_index("Category")
not_supported_variant_percent_evidence_df

,Percent of Accepted Variant Items,Percent of Not Accepted Variant Items
Category,,
Expression Variants,180 / 287 (62.72%),107 / 287 (37.28%)
Epigenetic Modification,14 / 14 (100.00%),0 / 14 (0.00%)
Fusion Variants,204 / 294 (69.39%),90 / 294 (30.61%)
Sequence Variants,73 / 133 (54.89%),60 / 133 (45.11%)
Gene Function Variants,49 / 91 (53.85%),42 / 91 (46.15%)
Rearrangement Variants,47 / 116 (40.52%),69 / 116 (59.48%)
Copy Number Variants,19 / 34 (55.88%),15 / 34 (44.12%)
Other Variants,42 / 83 (50.60%),41 / 83 (49.40%)
Genotype Variants,12 / 16 (75.00%),4 / 16 (25.00%)


In [72]:
summary_table_5 = not_supported_variant_percent_evidence_df

## Evidence Analysis

In [73]:
all_variant_evidence_df = pd.DataFrame(evidence_analysis_summary)

In [74]:
all_variant_evidence_df["Percentage of all CIViC Evidence Items"] = all_variant_evidence_df["Fraction of all CIViC Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of all CIViC Evidence Items"] + ")"
all_variant_evidence_df["Percentage of Accepted Evidence Items"] = all_variant_evidence_df["Fraction of Accepted Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of Accepted Evidence Items"] + ")"
all_variant_evidence_df["Percentage of Submitted Evidence Items"] = all_variant_evidence_df["Fraction of Submitted Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of Submitted Evidence Items"] + ")"

In [75]:
all_variant_evidence_df = all_variant_evidence_df.drop(["Fraction of all CIViC Evidence Items", "Fraction of Accepted Evidence Items", "Fraction of Submitted Evidence Items"], axis=1)

In [76]:
all_variant_evidence_percent_of_civic_df = all_variant_evidence_df.drop(["Percentage of Accepted Evidence Items", "Percentage of Submitted Evidence Items"], axis=1)
all_variant_evidence_percent_evidence_df = all_variant_evidence_df.drop(["Percentage of all CIViC Evidence Items"], axis=1)

Summary Table 6: The table below shows what percentage of all evidence items in CIViC are associated with Normalized, Unable to Normalize, and Not Supported variants. This will not add up to 100% because evidence itmes may be used across multiple variants.

In [77]:
all_variant_evidence_percent_of_civic_df = all_variant_evidence_percent_of_civic_df.set_index("Variant Category")
all_variant_evidence_percent_of_civic_df

,Percentage of all CIViC Evidence Items
Variant Category,
Normalized,6005 / 9920 (60.53%)
Unable to Normalize,129 / 9920 (1.30%)
Not Supported,4340 / 9920 (43.75%)


In [78]:
summary_table_6 = all_variant_evidence_percent_of_civic_df

Summmary Table 7: The table below shows the percentage of accepted and sumbitted evidence items per category of variants. 

In [79]:
all_variant_evidence_percent_evidence_df = all_variant_evidence_percent_evidence_df.set_index("Variant Category")
all_variant_evidence_percent_evidence_df

,Percentage of Accepted Evidence Items,Percentage of Submitted Evidence Items
Variant Category,,
Normalized,2080 / 6005 (34.64%),3925 / 6005 (65.36%)
Unable to Normalize,17 / 129 (13.18%),112 / 129 (86.82%)
Not Supported,2227 / 4340 (51.31%),2113 / 4340 (48.69%)


In [80]:
summary_table_7 = all_variant_evidence_percent_evidence_df

In [81]:
not_supported_variant_evidence_df = pd.DataFrame(not_supported_variant_evidence_summary)

In [82]:
not_supported_variant_evidence_df["Percent of CIViC Evidence Items"] = not_supported_variant_evidence_df["Fraction of CIViC Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of CIViC Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Not Supported Variant Evidence Items"] = not_supported_variant_evidence_df["Fraction of Not Supported Variant Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Not Supported Variant Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Accepted Evidence Items"] = not_supported_variant_evidence_df["Fraction of Accepted Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Accepted Evidence Items"] + ")"
not_supported_variant_evidence_df["Percent of Submitted Evidence Items"] = not_supported_variant_evidence_df["Fraction of Submitted Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of Submitted Evidence Items"] + ")"

In [83]:
not_supported_variant_evidence_df = not_supported_variant_evidence_df.drop(["Fraction of CIViC Evidence Items", "Fraction of Not Supported Variant Evidence Items", "Fraction of Accepted Evidence Items", "Fraction of Submitted Evidence Items"], axis=1)

In [84]:
not_supported_variant_evidence_percent_of_civic_df = not_supported_variant_evidence_df.drop(["Percent of Not Supported Variant Evidence Items", "Percent of Accepted Evidence Items", "Percent of Submitted Evidence Items"], axis=1)
not_supported_variant_evidence_percent_of_not_supported_df = not_supported_variant_evidence_df.drop(["Percent of CIViC Evidence Items", "Percent of Accepted Evidence Items", "Percent of Submitted Evidence Items"], axis=1)
not_supported_variant_evidence_percent_evidence_df = not_supported_variant_evidence_df.drop(["Percent of Not Supported Variant Evidence Items", "Percent of CIViC Evidence Items"], axis=1)

Summary Table 8: The table below shows the percentage of all CIViC evidence items that are associated with a Not Supported variant sub category. This will not add up to 100% since the evidence items can be associated with multiple variants.

In [85]:
not_supported_variant_evidence_percent_of_civic_df = not_supported_variant_evidence_percent_of_civic_df.set_index("Category")
not_supported_variant_evidence_percent_of_civic_df

,Percent of CIViC Evidence Items
Category,
Expression Variants,611 / 9920 (6.16%)
Epigenetic Modification,23 / 9920 (0.23%)
Fusion Variants,1247 / 9920 (12.57%)
Sequence Variants,308 / 9920 (3.10%)
Gene Function Variants,354 / 9920 (3.57%)
Rearrangement Variants,534 / 9920 (5.38%)
Copy Number Variants,63 / 9920 (0.64%)
Other Variants,145 / 9920 (1.46%)
Genotype Variants,29 / 9920 (0.29%)


In [86]:
summary_table_8 = not_supported_variant_evidence_percent_of_civic_df

Summary Table 9: The table below shows the percentage of all evidence items associated with Not Supported variants that are associated with a variant sub category. 

In [87]:
not_supported_variant_evidence_percent_of_not_supported_df = not_supported_variant_evidence_percent_of_not_supported_df.set_index("Category")
not_supported_variant_evidence_percent_of_not_supported_df

,Percent of Not Supported Variant Evidence Items
Category,
Expression Variants,611 / 4340 (14.08%)
Epigenetic Modification,23 / 4340 (0.53%)
Fusion Variants,1247 / 4340 (28.73%)
Sequence Variants,308 / 4340 (7.10%)
Gene Function Variants,354 / 4340 (8.16%)
Rearrangement Variants,534 / 4340 (12.30%)
Copy Number Variants,63 / 4340 (1.45%)
Other Variants,145 / 4340 (3.34%)
Genotype Variants,29 / 4340 (0.67%)


In [88]:
summary_table_9 = not_supported_variant_evidence_percent_of_not_supported_df

Summary Table 10: The table below shows the percentage of evidence items associated with Not Supported variant sub categories that are accepted or submitted.

In [89]:
not_supported_variant_evidence_percent_evidence_df = not_supported_variant_evidence_percent_evidence_df.set_index("Category")
not_supported_variant_evidence_percent_evidence_df

,Percent of Accepted Evidence Items,Percent of Submitted Evidence Items
Category,,
Expression Variants,342 / 611 (55.97%),269 / 611 (44.03%)
Epigenetic Modification,22 / 23 (95.65%),1 / 23 (4.35%)
Fusion Variants,747 / 1247 (59.90%),500 / 1247 (40.10%)
Sequence Variants,196 / 308 (63.64%),112 / 308 (36.36%)
Gene Function Variants,154 / 354 (43.50%),200 / 354 (56.50%)
Rearrangement Variants,202 / 534 (37.83%),332 / 534 (62.17%)
Copy Number Variants,27 / 63 (42.86%),36 / 63 (57.14%)
Other Variants,58 / 145 (40.00%),87 / 145 (60.00%)
Genotype Variants,17 / 29 (58.62%),12 / 29 (41.38%)


In [90]:
summary_table_10 = not_supported_variant_evidence_percent_evidence_df

## Impact

The bar graph below shows the relationship between the Not Suported variant sub category impact score and the sub category. Additionally, the colors illustrate the number of evidence items associated each sub category.

In [91]:
fig = px.bar(
    not_supported_variant_impact_df,
    x="Category",
    y="Total Sum Impact Score",
    hover_data=["Total Number Evidence Items", not_supported_variant_impact_df["% Accepted Evidence Items"]],
    color="Total Number Evidence Items",
    labels={"Total Sum Impact Score": "Total Sum Impact Score"},
    text_auto=".1f",
    color_continuous_scale="geyser"
)
fig.update_traces(width=1)
fig.show()

In [92]:
fig.write_html("civic_ns_categories_impact_redgreen.html")

The scatterplot below shows the relationship between the Not Suported variant sub category impact score and the number of evidence items associated with variants in each sub category. Additionally, the sizes of the data point represent the number of variants in each sub category. 

In [93]:
fig2 = px.scatter(
    data_frame=not_supported_variant_impact_df,
    x="Total Number Evidence Items",
    y="Total Sum Impact Score",
    size="Total Number Variants",
    size_max=40,
    text="Total Number Variants",
    color="Category",
    hover_data="% Accepted Evidence Items"
)
fig2.show()

In [94]:
fig2.write_html("civic_ns_categories_impact_scatterplot.html")